# Pruebas estadísticas de BFS, BFHS, IDS e IDHS

## Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from itertools import combinations

## Lectura de archivos

In [ ]:
# Diccionario de archivos y nombres.
files = {
    "BFHS": "Datos-BFHS.csv",
    "BFS": "Datos-BFS.csv",
    "IDHS": "Datos-IDHS.csv",
    "IDS": "Datos-IDS.csv"
}

# Al exportar los CSVs desde Excel, usó el ; para separar las columnas.
data = {name: pd.read_csv(path, sep=";", encoding="latin1") for name, path in files.items()}

### Manejo de tipo de datos de la tabla

In [ ]:
# Asegurar que las columnas numéricas sí lo sean (por si acaso).
for df in data.values():
    for col in metrics:
        df[col] = pd.to_numeric(df[col], errors="coerce")

## Análisis de métricas

In [ ]:
# Definir métricas
metrics = ["Tiempo (s)", "Memoria (KB)", "Profundidad"]

for metric in metrics:
    print("\n-------------------")
    print(f"Métrica: {metric}")
    print("-------------------")

    # Extraer los datos de cada algoritmo.
    arrays = [df[metric].dropna() for df in data.values()]

    # 1. Normalidad (Shapiro-Wilk)
    for name, df in data.items():
        stat, p = stats.shapiro(df[metric].dropna())
        print(f"Shapiro {name}: p = {p:.4g}")

    # 2. Homogeneidad de varianzas (Levene)
    stat, p = stats.levene(*arrays)
    print(f"Levene: p = {p:.4g}")

    # 3. ANOVA
    stat, p = stats.f_oneway(*arrays)
    print(f"ANOVA: p = {p:.4g}")

    # 4. Kruskal-Wallis
    stat, p = stats.kruskal(*arrays)
    print(f"Kruskal-Wallis: p = {p:.4g}")

    # 5. POST-HOC: Mann-Whitney U por pares (Bonferroni).
    # Esta prueba la aplicamos para ver entre cuáles algoritmos hay diferencias.
    # Como no se cumplen los supuestos de normalidad, debemos aplicar Mann-Whitney U
    # en lugar de t-test.
    # Además, debemos aplicar Bonferroni, pues al hacer varias pruebas (6 pares con 4 algoritmos),
    # sube el riesgo de falsos positivos.
    pairs = list(combinations(list(data.keys()), 2))
    m = len(pairs)
    results = []

    # Para cada par, tomamos la métrica actual y quitamos nulos.
    for a, b in pairs:
        x = data[a][metric].dropna()
        y = data[b][metric].dropna()
        if len(x) == 0 or len(y) == 0:
            results.append((a, b, np.nan, np.nan, np.nan, "—"))
            continue

        # Hay que recordar que H0: las distribuciones son iguales.
        U, p_raw = stats.mannwhitneyu(x, y, alternative="two-sided")
        p_bonf = min(p_raw * m, 1.0)

        # Solamente si p_bonf < 0.05, marcamos el par como significativo.
        sig = "Sí" if p_bonf < 0.05 else "No"
        results.append((a, b, U, p_raw, p_bonf, sig))

    # Ordenar por p ajustado ascendente
    results.sort(key=lambda r: (np.inf if pd.isna(r[4]) else r[4]))

    print("Post-hoc (Mann-Whitney U + Bonferroni):")
    print(f"{'Par':<15} {'U':>10} {'p':>12} {'p_bonf':>12} {'Sig':>5}")
    for a, b, U, p_raw, p_bonf, sig in results:
        if pd.isna(U):
            print(f"{a} vs {b:<7} {'-':>10} {'-':>12} {'-':>12} {sig:>5}")
        else:
            print(f"{a} vs {b:<7} {U:10.0f} {p_raw:12.3e} {p_bonf:12.3e} {sig:>5}")


-------------------
Métrica: Tiempo (s)
-------------------
Shapiro BFHS: p = 0.6034
Shapiro BFS: p = 0.3355
Shapiro IDHS: p = 0.3846
Shapiro IDS: p = 0.002492
Levene: p = 7.388e-09
ANOVA: p = 2.962e-20
Kruskal-Wallis: p = 2.083e-14
Post-hoc (Mann-Whitney U + Bonferroni):
Par                      U            p       p_bonf   Sig
BFHS vs IDS              0    6.796e-08    4.077e-07    Sí
IDHS vs IDS              0    6.796e-08    4.077e-07    Sí
BFS vs IDHS           399    7.898e-08    4.739e-07    Sí
BFS vs IDS             20    1.201e-06    7.205e-06    Sí
BFHS vs BFS             30    4.540e-06    2.724e-05    Sí
BFHS vs IDHS           349    5.896e-05    3.538e-04    Sí

-------------------
Métrica: Memoria (KB)
-------------------
Shapiro BFHS: p = 6.004e-06
Shapiro BFS: p = 2.535e-06
Shapiro IDHS: p = 8.32e-05
Shapiro IDS: p = 2.765e-05
Levene: p = 0.001182
ANOVA: p = 1.268e-23
Kruskal-Wallis: p = 1.046e-08
Post-hoc (Mann-Whitney U + Bonferroni):
Par                      U     